In [1]:
 import numpy as np

In [8]:
from numba import jit

In [35]:
from scipy import integrate

In [125]:
import numpy as np
from numba import jit
@jit(nopython=True)
def Numerov(f,u,x0,dx,h):
    """calculate the second order linear differential equation:
    x''(t)=f(t)x(t)+u(t),
    the input f,u are 1D array in numpy, they must be the same length.
    x0 = x(t=0), and dx = dx/dt(t=0)
    h is the spacing of t, the grid need be equal-spaced.
    """     
    x=np.zeros(len(f))
    
    x[0]=x0
    x[1]=x0+dx*h
    h2=h*h
    h12=h2/12
    w0=x[0]*(1-h12*f[0])-h12*u[0]
    w1=x[1]*(1-h12*f[1])-h12*u[1]

    for i in range(2,len(f)):
        w2=2*w1-w0+h2*f[i-1]*x[i-1]+u[i-1]*h2
        x[i]=(w2+h12*u[i])/(1-h12*f[i])
        (w0,w1)=(w1,w2)
              
    return x

In [132]:
def get_f(l,r,E,V_KS):
    """
    r must be array in numpy
    """
    return l*(l+1)/r**2+V_KS-E

插值的个数或许可以改变，调节，有数据之后可以尝试

In [141]:
from scipy import interpolate
#help(interpolate.UnivariateSpline)
def get_approx_u_0(u,r):
    """
    """
    spl=interpolate.UnivariateSpline(r[:4],u[:4],k=3,s=0)
    u_0=spl(0)
    return u_0

In [ ]:
def get_bound_state(E_range,l,V_KS,r,num_max):
    """
    """
    band=[]
    f=get_f(l,r,E,V_KS)
    u=Numerov(f,np.zeros(len(f)),0,-1e-5,r[1]-r[0])
    u_0=get_approx_u_0(u,r)
    for E in E_range:
        f=get_f(l,r,E,V_KS)
        u=Numerov(f,np.zeros(len(f)),0,-1e-5,r[1]-r[0])
        u_0=get_approx_u_0(u,r)
    
    
    return band